In [1]:
import pandas as pd
import numpy as np

In [2]:
# Concat all week csv files
import glob
import os
extension = 'csv'
all_filenames = [i for i in glob.glob("nfl-big-data-bowl-2021/week*.csv")]
weeks = pd.concat([pd.read_csv(f) for f in all_filenames ])

In [3]:
players = pd.read_csv('nfl-big-data-bowl-2021/players.csv')
games = pd.read_csv('nfl-big-data-bowl-2021/games.csv')
plays = pd.read_csv('nfl-big-data-bowl-2021/plays.csv')

In [4]:
weeks_merged = weeks.merge(games, on='gameId')
weeks_merged = weeks_merged.merge(plays, on=['gameId','playId'])
weeks_merged.head()

,time,x,y,s,a,dis,o,dir,event,nflId,...,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018-09-07T01:07:14.599Z,91.73,26.67,0.00,0.01,0.02,289.57,240.93,None,310.0,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
1,2018-09-07T01:07:14.599Z,88.89,36.47,0.01,0.01,0.01,105.63,66.66,None,79848.0,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
2,2018-09-07T01:07:14.599Z,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
3,2018-09-07T01:07:14.599Z,86.31,22.01,0.09,0.42,0.01,70.12,168.91,None,2495613.0,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
4,2018-09-07T01:07:14.599Z,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False


In [5]:
# rename column
weeks_merged.rename(columns={"frameId": "frame"})
# add homeTeamFlag dummy variable
weeks_merged['homeTeamFlag'] = np.where(weeks_merged['team']== 'home', 1, 0)
# add team column 
weeks_merged['teamAbbr'] = np.where(weeks_merged['homeTeamFlag']== 1, \
                                    weeks_merged['homeTeamAbbr'], weeks_merged['visitorTeamAbbr'])

In [6]:
position_dict = {'QB':'QB','SS':'DB', 'WR':'WR', 'FS':'DB', 'RB':'RB', 'MLB':'LB', 'CB':'DB', 'TE':'TE', \
                 'LB':'LB', 'FB':'RB', 'OLB':'LB','ILB':'LB','DB':'DB','HB':'RB','DL':'DL','NT':'DL', 'S':'DB', \
                'DE':'DL','P':'P','LS':'LS','K':'K', 'DT':'DL'}

# Mapping the dictionary keys to positionGroup
weeks_merged['positionGroup'] = weeks_merged['position'].map(position_dict) 
weeks_merged.head()

,time,x,y,s,a,dis,o,dir,event,nflId,...,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI,homeTeamFlag,teamAbbr,positionGroup
0,2018-09-07T01:07:14.599Z,91.73,26.67,0.00,0.01,0.02,289.57,240.93,None,310.0,...,NaN,NaN,C,10,10,0.261827,False,0,ATL,QB
1,2018-09-07T01:07:14.599Z,88.89,36.47,0.01,0.01,0.01,105.63,66.66,None,79848.0,...,NaN,NaN,C,10,10,0.261827,False,1,PHI,DB
2,2018-09-07T01:07:14.599Z,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,...,NaN,NaN,C,10,10,0.261827,False,0,ATL,WR
3,2018-09-07T01:07:14.599Z,86.31,22.01,0.09,0.42,0.01,70.12,168.91,None,2495613.0,...,NaN,NaN,C,10,10,0.261827,False,1,PHI,DB
4,2018-09-07T01:07:14.599Z,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,...,NaN,NaN,C,10,10,0.261827,False,0,ATL,WR


In [8]:
# For each team, export a csv that contains all data for which that team is on offense
teams = list(games.homeTeamAbbr.unique())
for team in teams:
    df = weeks_merged[weeks_merged['possessionTeam']==team]
    df.to_csv('additional_data/team_plays2'+team+'_plays.csv')
    print(team,'data written to', team+'_plays.csv')

Team identifier:
['PHI', 'CLE', 'IND', 'MIA', 'BAL', 'NE', 'NYG', 'NO', 'ARI', 'CAR', 'GB', 'DET', 'OAK', 'CIN', 'NYJ', 'BUF', 'ATL', 'PIT', 'WAS', 'TEN', 'TB', 'SF', 'LA', 'DEN', 'JAX', 'DAL', 'CHI', 'HOU', 'KC', 'MIN', 'SEA', 'LAC']
PHI data written to PHI_plays.csv
CLE data written to CLE_plays.csv
IND data written to IND_plays.csv
MIA data written to MIA_plays.csv
BAL data written to BAL_plays.csv
NE data written to NE_plays.csv
NYG data written to NYG_plays.csv
NO data written to NO_plays.csv
ARI data written to ARI_plays.csv
CAR data written to CAR_plays.csv
GB data written to GB_plays.csv
DET data written to DET_plays.csv
OAK data written to OAK_plays.csv
CIN data written to CIN_plays.csv
NYJ data written to NYJ_plays.csv
BUF data written to BUF_plays.csv
ATL data written to ATL_plays.csv
PIT data written to PIT_plays.csv
WAS data written to WAS_plays.csv
TEN data written to TEN_plays.csv
TB data written to TB_plays.csv
SF data written to SF_plays.csv
LA data written to LA_plays